

# MONITORAMENTO DE AÇÕES 


## install

In [0]:
!pip install requests
!pip install yfinance

You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-da4c2b9b-9cdc-4498-92d1-cf379639e85a/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 100 kB 4.4 MB/s 
[

*** WARNING: max output size exceeded, skipping output. ***

     |████████████████████████████████| 2.9 MB 13.7 MB/s 
  Installing build dependencies ... - \ | / done
  Getting requirements to build wheel ... - error
  ERROR: Command errored out with exit status 1:
   command: /local_disk0/.ephemeral_nfs/envs/pythonEnv-da4c2b9b-9cdc-4498-92d1-cf379639e85a/bin/python /local_disk0/.ephemeral_nfs/envs/pythonEnv-da4c2b9b-9cdc-4498-92d1-cf379639e85a/lib/python3.9/site-packages/pip/_vendor/pep517/in_process/_in_process.py get_requires_for_build_wheel /tmp/tmpgqnw955z
       cwd: /tmp/pip-install-24psh9ad/peewee_f06578034a8c4d1ebf37058f699018aa
  Complete output (40 lines):
  
  Error compiling Cython file:
  -----------------------------------------------

## import

In [0]:
import yfinance as yf
import pandas as pd
import requests
import json

## Config

In [0]:
url = "https://api.telegram.org/bot{token}/sendMessage"


config = [{
    "acao" : "WEGE3.SA",
    "percent_caiu" : 1,
    "percent_subiu" : 1.02,
    "vl_caiu" : 0.15,
    "vl_subiu" : 0.07,
    "ids_chat" : [5456369393]
    },
    {
    "acao" : "POMO4.SA",
    "percent_caiu" : 1,
    "percent_subiu" : 1.02,
    "vl_caiu" : 0.15,
    "vl_subiu" : 0.15,
    "ids_chat" : [5456369393]
    },
    {
    "acao" : "TUPY3.SA",
    "percent_caiu" : 1,
    "percent_subiu" : 1.02,
    "vl_caiu" : 0.15,
    "vl_subiu" : 0.15,
    "ids_chat" : [5456369393]  
    },
    {
    "acao" : "RAPT4.SA",
    "percent_caiu" : 1,
    "percent_subiu" : 1.02,
    "vl_caiu" : 0.15,
    "vl_subiu" : 0.15,
    "ids_chat" : [5456369393]
    }          
]


## Get ticket

In [0]:
def get_ticket(p_acao):
    '''
    função que retorna os valores de uma determinada ação na ultima cotação disponível 
    Args: 
        p_acao::string
    Returns:        
        vl_abertura:: float
        vl_fechamento:: float
        vl_variacao::float
        percent_variacao::float
    '''
    
    df = yf.download(p_acao,period='1d')
    
    vl_open = df.iloc[0]['Open']
    vl_close = df.iloc[0]['Close']
    
    #TRATA ERRO NO YFINANCE
    if(vl_open == 0 or vl_close == 0):
        print('problema no yfinance')
        df = yf.download(p_acao,period='2d')
        vl_open = df.iloc[0]['Open']
        vl_close = df.iloc[0]['Close']    



    vl_variacao = vl_close-vl_open    
    pc_variacao = ((vl_close*100)/vl_open)-100 

    return {"vl_abertura":round(vl_open,2), "vl_fechamento": round(vl_close,2),"vl_variacao":round(vl_variacao,2), "percent_variacao" : round(pc_variacao,2)}



## send message

In [0]:


def send_message_telegram(p_idChat, p_text):
    '''
    Envia mensagem para o Telegram - via avisa_acoes_bot

    Args:
        p_idChat::list -> Código unico da conversa
        p_text::String -> Corpo da mensagem
    '''

    for idchat in p_idChat:

        payload = json.dumps({
        "chat_id": idchat ,
        "text": p_text,
        "parse_mode" :"HTML"
        })
        headers = {
        'Content-Type': 'application/json'
        }
        response = requests.request("POST", url, headers=headers, data=payload)

    return True



## Flow

In [0]:


for trigger_alerta in config:
    print(trigger_alerta)

    # variaveis do gatilho
    acao = trigger_alerta.get("acao","")
    trigger_percent_caiu = trigger_alerta.get("percent_caiu",0)
    trigger_percent_subiu = trigger_alerta.get("percent_subiu",0)
    trigger_vl_caiu = trigger_alerta.get("vl_caiu",0)
    trigger_vl_subiu = trigger_alerta.get("vl_subiu",0)
    trigger_ids_chat = trigger_alerta.get("ids_chat",[])


    # busca cotação e valores
    json_cotacao = get_ticket(acao)

    real_vl_abertura = json_cotacao.get('vl_abertura',0)
    real_vl_fechamento = json_cotacao.get('vl_fechamento',0)
    real_vl_variacao = json_cotacao.get('vl_variacao',0)
    real_percent_variacao = json_cotacao.get('percent_variacao',0)

    
    #inicia validações
    
    print('############################')

    # VALIDA SE A AÇÃO CAIU EM %
    if trigger_percent_caiu != 0:
        print(f'Vai validar se a ação {acao} caiu {trigger_percent_caiu}%')
        print(f'A ação {acao} variou {real_percent_variacao}%')

        print(trigger_percent_caiu*-1)
        print(real_percent_variacao)
        
        if (trigger_percent_caiu*-1 >= real_percent_variacao):
        
            print('--->>> Gatilho ativado')
                    
            mensagem = f'''
                <b>ATENÇÃO</b>
                <i>avisa_acoes BOT Informa</i>

                <u>Sua ação <b>{acao}</b> caiu <b>{real_percent_variacao}%</b>  </u> 
                <pre>Trigger ->{trigger_percent_caiu*-1} % de queda </pre>
            
            '''


            send_message_telegram(trigger_ids_chat,mensagem)
            # se disparou uma mensagem por gatilho, não valida os demais gatilhos
            continue


    print('############################')

    # VALIDA SE A AÇÃO SUBIU EM %
    if trigger_percent_subiu != 0:
        print(f'Vai validar se a ação {acao} subiu {trigger_percent_subiu}%')
        print(f'A ação {acao} variou {real_percent_variacao}%')

        print(trigger_percent_subiu)
        print(real_percent_variacao)
        
        if (trigger_percent_subiu <= real_percent_variacao):
        
            print('--->>> Gatilho ativado')
        
            mensagem = f'''
                <b>ATENÇÃO</b>
                <i>avisa_acoes BOT Informa</i>

                <u>Sua ação <b>{acao}</b> subiu <b>{real_percent_variacao}%</b>  </u> 
                <pre>Trigger ->{trigger_percent_subiu} % de subida </pre>
            
            '''

            send_message_telegram(trigger_ids_chat,mensagem)   
            # se disparou uma mensagem por gatilho, não valida os demais gatilhos
            continue      

    # VALIDA SE A AÇÃO SUBIU EM VALOR
    if trigger_vl_subiu != 0:
        print(f'Vai validar se a ação {acao} subiu R${trigger_vl_subiu}')
        print(f'A ação {acao} variou {real_vl_variacao}%')

        print(trigger_vl_subiu)
        print(real_vl_variacao)
        
        if (trigger_vl_subiu <= real_vl_variacao):
        
            print('--->>> Gatilho ativado')
        
            mensagem = f'''
                <b>ATENÇÃO</b>
                <i>avisa_acoes BOT Informa</i>

                <u>Sua ação <b>{acao}</b> subiu <b> R$ {real_vl_variacao}</b>  </u> 
                <pre>Trigger ->R$ {trigger_vl_subiu} de subida </pre>
            
            '''

            send_message_telegram(trigger_ids_chat,mensagem)   
            # se disparou uma mensagem por gatilho, não valida os demais gatilhos
            continue                      

    # VALIDA SE A AÇÃO CAIU EM VALOR
    if trigger_vl_caiu != 0:
        print(f'Vai validar se a ação {acao} caiu R${trigger_vl_caiu}')
        print(f'A ação {acao} variou {real_vl_variacao}%')

        print(trigger_vl_caiu*-1)
        print(real_vl_variacao)
        
        if (trigger_vl_caiu*-1 >= real_vl_variacao):
        
            print('--->>> Gatilho ativado')

            mensagem = f'''
                <b>ATENÇÃO</b>
                <i>avisa_acoes BOT Informa</i>

                <u>Sua ação <b>{acao}</b> caiu <b> R$ {real_vl_variacao}</b>  </u> 
                <pre>Trigger ->R$ {trigger_vl_caiu*-1} de queda </pre>
            
            '''
        
            send_message_telegram(trigger_ids_chat,mensagem)         
            # se disparou uma mensagem por gatilho, não valida os demais gatilhos
            continue                                  



{'acao': 'WEGE3.SA', 'percent_caiu': 1, 'percent_subiu': 1.02, 'vl_caiu': 0.15, 'vl_subiu': 0.07, 'ids_chat': [5456369393]}
[*********************100%%**********************]  1 of 1 completed
############################
Vai validar se a ação WEGE3.SA caiu 1%
A ação WEGE3.SA variou -2.33%
-1
-2.33
--->>> Gatilho ativado
{'acao': 'POMO4.SA', 'percent_caiu': 1, 'percent_subiu': 1.02, 'vl_caiu': 0.15, 'vl_subiu': 0.15, 'ids_chat': [5456369393]}
[*********************100%%**********************]  1 of 1 completed
############################
Vai validar se a ação POMO4.SA caiu 1%
A ação POMO4.SA variou 0.0%
-1
0.0
############################
Vai validar se a ação POMO4.SA subiu 1.02%
A ação POMO4.SA variou 0.0%
1.02
0.0
Vai validar se a ação POMO4.SA subiu R$0.15
A ação POMO4.SA variou 0.0%
0.15
0.0
Vai validar se a ação POMO4.SA caiu R$0.15
A ação POMO4.SA variou 0.0%
-0.15
0.0
{'acao': 'TUPY3.SA', 'percent_caiu': 1, 'percent_subiu': 1.02, 'vl_caiu': 0.15, 'vl_subiu': 0.15, 'ids_chat': 